**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2021*

----

# Trabajo práctico entregable - parte 2


En el ejercicio 1 de la parte 1 del entregable seleccionaron las filas y columnas relevantes al problema de predicción de precios de una propiedad. Además de ello, tuvieron que reducir el número de valores posibles para las variables categóricas utilizando información de dominio.

En el ejercicio 2 de la parte 1 del entregable imputaron los valores faltantes de las columnas `Suburb` y las columnas obtenidas a partir del conjunto de datos `airbnb`.

En esta notebook, **se utilizará resultado de dichas operaciones.**


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns
sns.set_context('talk')

In [2]:
# Acá deberían leer el conjunto de datos que ya tienen.
melb_df_origin = pd.read_csv('datasets/melb_df_firt_part.csv')
melb_df_origin[:3]

,Suburb,Rooms,Type,Price,Date,Distance,Bathroom,Car,CouncilArea,Postcode,Lattitude,Longtitude,Landsize,BuildingArea,YearBuilt,Regionname,AgeRange,price_mean_city
0,Abbotsford,2,h,1480000.0,2016-03-12,2.5,1.0,1.0,Yarra,3067,-37.7996,144.9984,202.0,NaN,NaN,North,S/D,121.34
1,Abbotsford,2,h,1035000.0,2016-04-02,2.5,1.0,0.0,Yarra,3067,-37.8079,144.9934,156.0,79.0,1900.0,North,[>70),121.34
2,Abbotsford,3,h,1465000.0,2017-04-03,2.5,2.0,0.0,Yarra,3067,-37.8093,144.9944,134.0,150.0,1900.0,North,[>70),121.34


In [3]:
melb_df_origin.shape

(12956, 18)

## Ejercicio 1: Encoding

1. Seleccionar todas las filas y columnas del conjunto de datos obtenido en la parte 1 del entregable, **excepto** `BuildingArea` y `YearBuilt`, que volveremos a imputar más adelante.

2. Aplicar una codificación One-hot encoding a cada fila, tanto para variables numéricas como categóricas. Si lo consideran necesario, pueden volver a reducir el número de categorías únicas.

Algunas opciones:
  1. Utilizar `OneHotEncoder` junto con el parámetro `categories` para las variables categóricas y luego usar `numpy.hstack` para concatenar el resultado con las variables numéricas. 
  2. `DictVectorizer` con algunos pasos de pre-proceso previo.

Recordar también que el atributo `pandas.DataFrame.values` permite acceder a la matriz de numpy subyacente a un DataFrame.

### Selección de columnas

In [4]:
melb_df = melb_df_origin.copy()
melb_df.head()

,Suburb,Rooms,Type,Price,Date,Distance,Bathroom,Car,CouncilArea,Postcode,Lattitude,Longtitude,Landsize,BuildingArea,YearBuilt,Regionname,AgeRange,price_mean_city
0,Abbotsford,2,h,1480000.0,2016-03-12,2.5,1.0,1.0,Yarra,3067,-37.7996,144.9984,202.0,NaN,NaN,North,S/D,121.34
1,Abbotsford,2,h,1035000.0,2016-04-02,2.5,1.0,0.0,Yarra,3067,-37.8079,144.9934,156.0,79.0,1900.0,North,[>70),121.34
2,Abbotsford,3,h,1465000.0,2017-04-03,2.5,2.0,0.0,Yarra,3067,-37.8093,144.9944,134.0,150.0,1900.0,North,[>70),121.34
3,Abbotsford,3,h,850000.0,2017-04-03,2.5,2.0,1.0,Yarra,3067,-37.7969,144.9969,94.0,NaN,NaN,North,S/D,121.34
4,Abbotsford,4,h,1600000.0,2016-04-06,2.5,1.0,2.0,Yarra,3067,-37.8072,144.9941,120.0,142.0,2014.0,North,[0-5),121.34


In [5]:
melb_df.drop(columns=['BuildingArea', 'YearBuilt'], inplace=True)

Llegados a este punto, concluimos que por la cantidad de valores y distribución en las columnas **Lattitude** y **Longtitude**, y por los valores conceptuales de las variables, las mismas no las usaremos dentro de nuestro análisis. Se volverán a agregar si fuesen necesarias.

In [6]:
melb_df.drop(columns=['Lattitude', 'Longtitude'], inplace=True)

In [7]:
melb_df.shape

(12956, 14)

### One-Hot Encoding

#### Rápido Análisis

In [8]:
melb_df.nunique()

Suburb              314
Rooms                 9
Type                  3
Price              1945
Date                 58
Distance            202
Bathroom              8
Car                  11
CouncilArea          33
Postcode            198
Landsize           1398
Regionname            4
AgeRange             16
price_mean_city      31
dtype: int64

In [9]:
melb_df.isna().sum()

Suburb             0
Rooms              0
Type               0
Price              0
Date               0
Distance           0
Bathroom           0
Car                0
CouncilArea        0
Postcode           0
Landsize           0
Regionname         0
AgeRange           0
price_mean_city    0
dtype: int64

In [10]:
melb_df.columns.values

array(['Suburb', 'Rooms', 'Type', 'Price', 'Date', 'Distance', 'Bathroom',
       'Car', 'CouncilArea', 'Postcode', 'Landsize', 'Regionname',
       'AgeRange', 'price_mean_city'], dtype=object)

#### Re-Agrupaciones

Como decidimos dejar la columna **Date**, la agruparemos para mostrar simplemente la fecha de venta pero dividido trimestralmente.

In [11]:
#df['quarter'] = pd.PeriodIndex(df.date, freq='Q')
melb_df['Date_Quarter'] = pd.PeriodIndex(melb_df.Date, freq='Q')
melb_df.Date_Quarter.unique()

<PeriodArray>
['2016Q1', '2016Q2', '2017Q2', '2016Q3', '2016Q4', '2017Q1', '2017Q3',
 '2017Q4']
Length: 8, dtype: period[Q-DEC]

Quedandonos con 8 categorías, borraremos la columna **Date**

In [12]:
melb_df.drop(columns=['Date'], inplace=True)
melb_df.head()

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,CouncilArea,Postcode,Landsize,Regionname,AgeRange,price_mean_city,Date_Quarter
0,Abbotsford,2,h,1480000.0,2.5,1.0,1.0,Yarra,3067,202.0,North,S/D,121.34,2016Q1
1,Abbotsford,2,h,1035000.0,2.5,1.0,0.0,Yarra,3067,156.0,North,[>70),121.34,2016Q2
2,Abbotsford,3,h,1465000.0,2.5,2.0,0.0,Yarra,3067,134.0,North,[>70),121.34,2017Q2
3,Abbotsford,3,h,850000.0,2.5,2.0,1.0,Yarra,3067,94.0,North,S/D,121.34,2017Q2
4,Abbotsford,4,h,1600000.0,2.5,1.0,2.0,Yarra,3067,120.0,North,[0-5),121.34,2016Q2


Dejaremos el resto de columnas como están, por el momento.

#### Encoding

Primero tomaremos aquellas variables categóricas y las numéricas

In [54]:
melb_df.columns.values

array(['Suburb', 'Rooms', 'Type', 'Price', 'Distance', 'Bathroom', 'Car',
       'CouncilArea', 'Postcode', 'Landsize', 'Regionname', 'AgeRange',
       'price_mean_city', 'Date_Quarter'], dtype=object)

In [55]:
categorical_cols = ['Suburb', 'Type', 'CouncilArea', 'Postcode', 'Regionname', 'Date_Quarter', 'AgeRange']
numerical_cols = ['Rooms', 'Price', 'Distance', 'Bathroom', 'Car', 'Landsize', 'price_mean_city']

Aplicamos OneHotEncoder sobre las variables categóricas

In [56]:
from sklearn.preprocessing import OneHotEncoder

new_cat_cols = []

encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X = encoder.fit_transform(melb_df[categorical_cols])
for col, col_values in zip(categorical_cols, encoder.categories_):
    for col_value in col_values:
        new_cat_cols.append('{}={}'.format(col, col_value))
# Hacemos un DataFrame con las columnas encodeadas
df_cat_enc = pd.DataFrame(X, columns = new_columns)

Unimos datasets en uno solo

In [57]:
melb_df_enc = np.hstack([melb_df[numerical_cols], df_cat_enc])
df_cat_enc = np.hstack([numerical_cols, new_cat_cols])

In [59]:
melb_df_enc.shape

(12956, 583)

In [61]:
pd.DataFrame(melb_df_enc, columns=df_cat_enc)

,Rooms,Price,Distance,Bathroom,Car,Landsize,price_mean_city,Suburb=Abbotsford,Suburb=Aberfeldie,Suburb=Airport West,...,AgeRange=[30-35),AgeRange=[35-40),AgeRange=[40-45),AgeRange=[45-50),AgeRange=[5-10),AgeRange=[50-55),AgeRange=[55-60),AgeRange=[60-65),AgeRange=[65-70),AgeRange=[>70)
0,2.0,1480000.0,2.5,1.0,1.0,202.0,121.34,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,1035000.0,2.5,1.0,0.0,156.0,121.34,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3.0,1465000.0,2.5,2.0,0.0,134.0,121.34,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3.0,850000.0,2.5,2.0,1.0,94.0,121.34,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,1600000.0,2.5,1.0,2.0,120.0,121.34,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12951,3.0,640000.0,16.5,2.0,2.0,607.0,91.22,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12952,3.0,366000.0,44.2,1.0,1.0,502.0,121.34,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12953,5.0,1355000.0,48.1,3.0,5.0,44500.0,121.34,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12954,4.0,625500.0,23.8,2.0,2.0,477.0,90.62,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## Ejercicio 2: Imputación por KNN

En el teórico se presentó el método `IterativeImputer` para imputar valores faltantes en variables numéricas. Sin embargo, los ejemplos presentados sólo utilizaban algunas variables numéricas presentes en el conjunto de datos. En este ejercicio, utilizaremos la matriz de datos codificada para imputar datos faltantes de manera más precisa.

1. Agregue a la matriz obtenida en el punto anterior las columnas `YearBuilt` y `BuildingArea`.
2. Aplique una instancia de `IterativeImputer` con un estimador `KNeighborsRegressor` para imputar los valores de las variables. ¿Es necesario estandarizar o escalar los datos previamente?
3. Realice un gráfico mostrando la distribución de cada variable antes de ser imputada, y con ambos métodos de imputación.

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import IterativeImputer

melb_data_mice = melb_df.copy(deep=True)

mice_imputer = IterativeImputer(random_state=0, estimator=KNeighborsRegressor())
melb_data_mice[['YearBuilt','BuildingArea']] = mice_imputer.fit_transform(
    melb_data_mice[['YearBuilt', 'BuildingArea']])

Ejemplo de gráfico comparando las distribuciones de datos obtenidas con cada método de imputación.

In [ ]:
mice_year_built = melb_data_mice.YearBuilt.to_frame()
mice_year_built['Imputation'] = 'KNN over YearBuilt and BuildingArea'
melb_year_build = melb_df.YearBuilt.dropna().to_frame()
melb_year_build['Imputation'] = 'Original'
data = pandas.concat([mice_year_built, melb_year_build])
fig = plt.figure(figsize=(8, 5))
g = seaborn.kdeplot(data=data, x='YearBuilt', hue='Imputation')

## Ejercicio 3: Reducción de dimensionalidad.

Utilizando la matriz obtenida en el ejercicio anterior:
1. Aplique `PCA` para obtener $n$ componentes principales de la matriz, donde `n = min(20, X.shape[0])`. ¿Es necesario estandarizar o escalar los datos?
2. Grafique la varianza capturada por los primeros $n$ componentes principales, para cada $n$.
3. En base al gráfico, seleccione las primeras $m$ columnas de la matriz transformada para agregar como nuevas características al conjunto de datos.

## Ejercicio 4: Composición del resultado

Transformar nuevamente el conjunto de datos procesado en un `pandas.DataFrame` y guardarlo en un archivo.

Para eso, será necesario recordar el nombre original de cada columna de la matriz, en el orden correcto. Tener en cuenta:
1. El método `OneHotEncoder.get_feature_names` o el atributo `OneHotEncoder.categories_` permiten obtener una lista con los valores de la categoría que le corresponde a cada índice de la matriz.
2. Ninguno de los métodos aplicados intercambia de lugar las columnas o las filas de la matriz.

In [ ]:
## Small example
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder

## If we process our data with the following steps:
categorical_cols = ['Type', 'Regionname']
numerical_cols = ['Rooms', 'Distance']
new_columns = []

# Step 1: encode categorical columns
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_cat = encoder.fit_transform(melb_df[categorical_cols])
for col, col_values in zip(categorical_cols, encoder.categories_):
  for col_value in col_values:
    new_columns.append('{}={}'.format(col, col_value))
print("Matrix has shape {}, with columns: {}".format(X_cat.shape, new_columns))

# Step 2: Append the numerical columns
X = numpy.hstack([X_cat, melb_df[numerical_cols].values])
new_columns.extend(numerical_cols)
print("Matrix has shape {}, with columns: {}".format(X_cat.shape, new_columns))

# Step 3: Append some new features, like PCA
pca = PCA(n_components=2)
pca_dummy_features = pca.fit_transform(X)
X_pca = numpy.hstack([X, pca_dummy_features])
new_columns.extend(['pca1', 'pca2'])

## Re-build dataframe
processed_melb_df = pandas.DataFrame(data=X_pca, columns=new_columns)
processed_melb_df.head()

## Ejercicio 5: Documentación

En un documento `.pdf` o `.md` realizar un reporte de las operaciones que realizaron para obtener el conjunto de datos final. Se debe incluir:
  1. Criterios de exclusión (o inclusión) de filas
  2. Interpretación de las columnas presentes
  2. Todas las transofrmaciones realizadas

Este documento es de uso técnico exclusivamente, y su objetivo es permitir que otres desarrolladores puedan reproducir los mismos pasos y obtener el mismo resultado. Debe ser detallado pero consiso. Por ejemplo:

```
  ## Criterios de exclusión de ejemplos
  1. Se eliminan ejemplos donde el año de construcción es previo a 1900

  ## Características seleccionadas
  ### Características categóricas
  1. Type: tipo de propiedad. 3 valores posibles
  2. ...
  Todas las características categóricas fueron codificadas con un
  método OneHotEncoding utilizando como máximo sus 30 valores más 
  frecuentes.
  
  ### Características numéricas
  1. Rooms: Cantidad de habitaciones
  2. Distance: Distancia al centro de la ciudad.
  3. airbnb_mean_price: Se agrega el precio promedio diario de 
     publicaciones de la plataforma AirBnB en el mismo código 
     postal. [Link al repositorio con datos externos].

  ### Transformaciones:
  1. Todas las características numéricas fueron estandarizadas.
  2. La columna `Suburb` fue imputada utilizando el método ...
  3. Las columnas `YearBuilt` y ... fueron imputadas utilizando el 
     algoritmo ...
  4. ...

  ### Datos aumentados
  1. Se agregan las 5 primeras columnas obtenidas a través del
     método de PCA, aplicado sobre el conjunto de datos
     totalmente procesado.
```
